In [2]:
from azureml.core import Workspace, Dataset, Experiment, Run
from azureml.core.model import Model
from azureml.core.compute import ComputeTarget

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib

In [3]:
# ======================================================
# Connect to Workspace
# ======================================================
ws = Workspace.from_config() 


In [5]:
# ===========================================================
# Load Dataset either through Azure ML or from local file
# ===========================================================
try:
    dataset = Dataset.get_by_name(ws, name='maintenance-data')
    df = dataset.to_pandas_dataframe()
    print("✅ Loaded dataset from Azure ML")
except Exception as e:
    print("⚠️ Dataset not found in workspace. Using local CSV instead.")
    df = pd.read_csv("iot_sensor_data.csv")

print(df.head())

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
✅ Loaded dataset from Azure ML
   temperature  vibration   pressure   voltage  hours_since_maintenance  \
0    79.967142   0.439936  26.624109  2.727658                      407   
1    73.617357   0.392463  29.277407  3.041884                       50   
2    81.476885   0.305963  26.037900  3.175918                      265   
3    90.230299   0.235306  28.460192  3.866306                      119   
4    72.658466   0.369822  20.531927  3.466966                      199   

   failure  
0        1  
1        0  
2        0  
3        0  
4        0  


/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/mlflow/__init__.py:41: UserWarning: Versions of mlflow (3.1.1) and mlflow-skinny (2.22.1) are different. This may lead to unexpected behavior. Please install the same version of both packages.
  mlflow.mismatch._check_version_mismatch()


In [6]:
# ======================================================
# Split Data
# ======================================================
X = df.drop("failure", axis=1)
y = df["failure"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# ======================================================
# Start Experiment Tracking
# ======================================================
experiment = Experiment(workspace=ws, name="predictive-maintenance-training")
run = experiment.start_logging()

In [8]:
# ======================================================
# Train Model
# ======================================================
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [9]:
# ======================================================
# Evaluate Model
# ======================================================
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

# Log metrics
run.log("accuracy", acc)
run.log("true_failures", int(y_test.sum()))
run.log("predicted_failures", int(y_pred.sum()))

print(f"✅ Accuracy: {acc:.3f}")
print("\nConfusion Matrix:\n", cm)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

✅ Accuracy: 0.965

Confusion Matrix:
 [[182   0]
 [  7  11]]

Classification Report:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98       182
           1       1.00      0.61      0.76        18

    accuracy                           0.96       200
   macro avg       0.98      0.81      0.87       200
weighted avg       0.97      0.96      0.96       200



In [10]:
# ======================================================
# Save and Register Model
# ======================================================
model_filename = "predictive_maintenance_model.pkl"
joblib.dump(model, model_filename)

registered_model = Model.register(workspace=ws,model_name='maintenance-predictor',model_path=model_filename)

Registering model maintenance-predictor


In [11]:
# ======================================================
# Manual registration
# ======================================================

from azureml.core import Model

model = Model.register(
    workspace=ws,
    model_name='maintenance-predictor',   
    model_path='predictive_maintenance_model.pkl',                
    description='Predictive maintenance model trained on IoT-like sensor data'
)

print("Model registered:", model.name, "version:", model.version)

Registering model maintenance-predictor
Model registered: maintenance-predictor version: 3


In [24]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig

env = Environment("maintenance-env")
deps = CondaDependencies.create(
    conda_packages=["scikit-learn", "numpy", "pandas"],
    pip_packages=["azureml-defaults"]
)
env.python.conda_dependencies = deps

inference_config = InferenceConfig(
    entry_script="score.py",
    environment=env
)

In [25]:
from azureml.core.webservice import AciWebservice, Webservice

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    auth_enabled=True
)

service = Model.deploy(
    workspace=ws,
    name="maintenance-endpoint",
    models=[model],
    inference_config=inference_config,
    deployment_config=deployment_config,
    overwrite=True
)

service.wait_for_deployment(show_output=True)

/tmp/ipykernel_2940/1189117819.py:9: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-10-22 08:40:34+00:00 Creating Container Registry if not exists.
2025-10-22 08:40:34+00:00 Registering the environment.
2025-10-22 08:40:36+00:00 Use the existing image.
2025-10-22 08:40:36+00:00 Generating deployment configuration.
2025-10-22 08:40:39+00:00 Submitting deployment to compute.
2025-10-22 08:40:45+00:00 Checking the status of deployment maintenance-endpoint..
2025-10-22 08:41:49+00:00 Checking the status of inference endpoint maintenance-endpoint.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [4]:
from azureml.core import Workspace
ws=Workspace.from_config()

In [5]:
# ======================================================================
# Checking existence
# ======================================================================
from azureml.core.webservice import Webservice

service = Webservice(name="maintenance-endpoint", workspace=ws)
print("Status:", service.state)
print("Scoring URI:", service.scoring_uri)
print("Authentication enabled:", service.auth_enabled)

Status: Healthy
Scoring URI: http://732bb5f1-0574-4fe0-a7a6-24b61136f99f.eastus.azurecontainer.io/score
Authentication enabled: True


In [6]:
key = service.get_keys()[0]
print("API Key:", key)

API Key: lexqRONwjJr5dAz6DohUEcHGodJ6shpx


In [7]:
# ============================================================
# Mimic IoT payload
# ===========================================================
import requests
import json

#Replace with your scoring URI
url = service.scoring_uri

#Replace with your actual API key
headers = {'Content-Type': 'application/json', 'Authorization': f'Bearer {key}'}

#Sample data (one record)
data = {
    "data": [[75.2, 0.22, 30.5, 220, 500]]
}

response = requests.post(url, headers=headers, data=json.dumps(data))

print("Response status:", response.status_code)
print("Prediction:", response.json())

Response status: 200
Prediction: {"predictions": [0]}


In [8]:
data = {
    "data": [
        [80.1, 0.45, 32.1, 210, 900],
        [72.5, 0.18, 28.3, 225, 300]
    ]
}

response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.json())

{"predictions": [0, 0]}
